In [ ]:
# Importing gradio for the UI
import gradio as gr


In [2]:
# Import the env_vars module
import sys
import os

from modules.env_vars import set_os_env_vars, check_missing_vars
from modules.neon_db import run_neon_query, load_sql_query
from modules.date_functions import get_current_date
from modules.reference_extraction import create_content_from_df
from modules.prompt_templates import one_shot_example, system_message_example

set_os_env_vars() # This will execute the code in env_vars.py and put the environment variables in os

In [3]:
from modules.langchain_config import set_langsmith_client, get_langsmith_tracer, get_llm_model, load_model_costs

set_langsmith_client()
tracer = get_langsmith_tracer()

In [ ]:
# Load the costs
MODEL_COSTS = load_model_costs()
MODEL_COSTS


In [5]:
# Initialize the language model
model_name = "claude-3-5-sonnet-20241022"
# model_name = "gpt-4o-mini"
streaming = True # Streaming is when the LLM returns a token at a time, instead of the entire response at once

# Initialize the language model
llm = get_llm_model(model_name, streaming, MODEL_COSTS)


In [ ]:
query = load_sql_query("web_pages.sql")
df = run_neon_query(query)

print("Number of rows:", len(df.index))
df.head(1)

In [ ]:
# Print out the results (summary, titles, etc.)
all_content, all_content_list = create_content_from_df(df)

print(len(all_content_list))
print(all_content_list[0])


In [ ]:
newsletter_prompt = one_shot_example(get_current_date())

# Create the chain with tracing
chain = (newsletter_prompt | llm).with_config(
    {
        "callbacks": [tracer],
        "tags": ["newsletter_generation"],
    }
)

# Test the chain
newsletter = chain.invoke({"context": all_content, "current_date": get_current_date()})
print(newsletter.content)

In [ ]:
newsletter_prompt = system_message_example()

# Create the chain with tracing
chain = (newsletter_prompt | llm).with_config(
    {
        "callbacks": [tracer],
        "tags": ["newsletter_generation"],
    }
)

# Test the chain
newsletter = chain.invoke({"context": all_content, "current_date": get_current_date()})
print(newsletter.content)